In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
url_dmv = 'https://raw.githubusercontent.com/kansjs/IBM_applied_capstone_share/master/DMV_grid2.csv'
dmv_grid = pd.read_csv(url_dmv)
dmv_grid.head()

,Lat1,Long1,Lat2,Long2
0,39.27,-76.710000,39.187143,-76.833333
1,39.27,-76.833333,39.187143,-76.956667
2,39.27,-76.956667,39.187143,-77.080000
3,39.27,-77.080000,39.187143,-77.203333
4,39.27,-77.203333,39.187143,-77.326667


In [3]:
dmv_grid.shape

(42, 4)

In [2]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [5]:
categoryId = '52dea92d3cf9994f4e043dbb'
LIMIT = 100

In [6]:
def getNearbyVenues(latitude1, longitude1, latitude2, longitude2):
    venues_list=[]
   
    for Lat1, Long1, Lat2, Long2 in zip(latitude1, longitude1, latitude2, longitude2):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&intent=browse&ne={},{}&sw={},{}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            Lat1, 
            Long1, 
            Lat2,
            Long2,
            LIMIT,
            categoryId)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Venue', 
                  'Latitude', 
                  'Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
Discount = getNearbyVenues(latitude1 = dmv_grid['Lat1'], 
                               longitude1 = dmv_grid['Long1'], 
                               latitude2 = dmv_grid['Lat2'], 
                               longitude2 = dmv_grid['Long2']
                                  )

In [8]:
Discount.head()

,Venue,Latitude,Longitude,Venue Category
0,Nordstrom Rack,39.195553,-76.811755,Discount Store
1,Linq Services,39.190040,-76.747227,Discount Store
2,Hobby Lobby,39.183617,-76.816762,Fabric Shop
3,Five Below,39.155372,-76.725045,Discount Store
4,Nike Factory Store,39.157145,-76.724887,Sporting Goods Shop


In [9]:
Discount.shape

(187, 4)

In [10]:
Discount.to_csv('Discount.2.9.20.csv', index=False)